# WSCC 9-bus System Static Test

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

WSCC-9bus_CIM ${TOP}/Examples/CIM/grid-data/WSCC-09/WSCC-09_RX/*.xml

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt

## Initialization

In [ ]:
path = 'logs/WSCC-9bus/'
logName = 'WSCC-9bus_InitLeftVector'
logFilename = path + logName + '.csv'
print(logFilename)

ts_dpsim_init = rt.read_timeseries_dpsim(logFilename)

### Phasors at first time step

In [ ]:
phasors_init = ts.phasors(ts_dpsim_init)

for node, phasor in phasors_init.items():
    print(node + ': ' + str(phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[0]))

### Phasors at last time step

In [ ]:
for node, phasor in phasors_init.items():
    print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

### Difference

In [ ]:
for node, phasor in phasors_init.items():
    print(node + ': ' + str(phasor['abs'].values[-1] - phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[-1] - phasor['phase'].values[0]))

## Simulation

In [ ]:
path = 'logs/WSCC-9bus/'
logName = 'WSCC-9bus'
logFilename = path + logName + '.csv'
print(logFilename)

ts_dpsim = rt.read_timeseries_dpsim(logFilename)
phasors = ts.phasors(ts_dpsim)

### Phasors at last time step in per unit

In [ ]:
nominal_voltages = {'v1': 16500, 'v2': 18000, 'v3': 13800, 
                         'v4': 230000, 'v5': 230000, 'v6': 230000, 
                         'v7': 230000, 'v8': 230000, 'v9': 230000} 
for node, nom_voltage in nominal_voltages.items():
    print(node + ': ' + str(phasors[node]['abs'].values[0] / nom_voltage) + '<' + str(phasors[node]['phase'].values[0]))

### Plot node phase
The phase at BUS9 (solution vector: node00007) is not initialized properly from the CIM powerflow data. Therefore, the steady-state initialization implemented in the MNA solver is executed first so that the initial phase at t=0 is already very close to the correct one.
Using PI-lines with parallel impedances to ground not only during the initialization but also the simulation or more accurate CIM load flow results would probably further improve the results because the values around BUS9 have still not converged completely.

In [ ]:
pt.plot_timeseries(1, phasors['v9']['phase'])
plt.ylim([2.10405, 2.1042])

The next plot is showing the same value, BUS9 phase during the initialization. The phase oscillation is larger than in the actual simulation.

In [ ]:
pt.plot_timeseries(1, phasors_init['n00007f00']['phase'])
plt.ylim([2.10405, 2.1042])

### Plot node voltages

In [ ]:
for node, phasor in phasors.items():
    print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

In [ ]:
pt.plot_timeseries(1, phasors['v1']['abs'])
pt.plot_timeseries(1, phasors['v2']['abs'])
pt.plot_timeseries(1, phasors['v3']['abs'])

pt.plot_timeseries(2, phasors['v4']['abs'])
pt.plot_timeseries(2, phasors['v5']['abs'])
pt.plot_timeseries(2, phasors['v6']['abs'])
pt.plot_timeseries(2, phasors['v7']['abs'])
pt.plot_timeseries(2, phasors['v8']['abs'])
pt.plot_timeseries(2, phasors['v9']['abs'])
plt.gca().axes.set_ylim([200000,240000])

### Assert Steady-State

In [ ]:
for node, phasor in phasors.items():
    abs_diff = phasor['abs'].values[-1] - phasor['abs'].values[0]
    assert abs_diff < 20
    phase_diff = phasor['phase'].values[-1] - phasor['phase'].values[0]
    assert phase_diff < 0.001
    print(node + ': ' + str(abs_diff) + '<' + str(phase_diff))